In [2]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

client = OpenAI()

## Upload file for retrieval

In [17]:
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose="assistants"
)

dict(file)

{'id': 'file-ih0PSuryOrEiVN4X5jZq6TOv',
 'bytes': 48802,
 'created_at': 1703524662,
 'filename': 'zia_profile.pdf',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

## Enable Retrieval in Assistant

In [18]:
# Attach file to assistant
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}], # enable retrieval tool
  file_ids=[file.id] # attach file id
)

dict(assistant)

{'id': 'asst_8C3QlObReJVGDnRZ1NaUZCev',
 'created_at': 1703524665,
 'description': None,
 'file_ids': ['file-ih0PSuryOrEiVN4X5jZq6TOv'],
 'instructions': 'You are a customer support chatbot. Use your knowledge base to best respond to customer queries.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolRetrieval(type='retrieval')]}

## Thread and Message

In [19]:
# Create a thread
thread = client.beta.threads.create()

# Attach a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the date of birth of Zia?"
)

dict(message)


{'id': 'msg_XCxn9YPjhAFfV0wW4vkdyzNd',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What is the date of birth of Zia?'), type='text')],
 'created_at': 1703524688,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_niFbXQXEuJxPAANsjxSJXzKk'}

## Run

In [12]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

dict(run)

{'id': 'run_s50aAZC3KSkOaEvXawC3z1KS',
 'assistant_id': 'asst_6B9XYalRKIWXXWjpmFsIIiH1',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703524506,
 'expires_at': 1703525106,
 'failed_at': None,
 'file_ids': ['file-pk7HMOhF3x4LQCWHWnA08fz1'],
 'instructions': 'You are a customer support chatbot. Use your knowledge base to best respond to customer queries.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_1lstmo59ECaxNsaicn2zlDsu',
 'tools': [ToolAssistantToolsRetrieval(type='retrieval')]}

## Response

In [13]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(messages)

print("")
# displaying in conversation style
for message in reversed(messages.data):
  # revesed() because assistant message was appended and it is listed first.
  print(message.role + ": " + message.content[0].text.value)


SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_zRvM9RxZT563vESiMwMvleip', assistant_id='asst_6B9XYalRKIWXXWjpmFsIIiH1', content=[MessageContentText(text=Text(annotations=[TextAnnotationFileCitation(end_index=46, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-pk7HMOhF3x4LQCWHWnA08fz1', quote='Zia Khan was born in Sialkot in 1961 in an army garrison. His father the late Lt'), start_index=36, text='【7†source】', type='file_citation')], value='Zia Khan was born in Sialkot in 1961【7†source】.'), type='text')], created_at=1703524510, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_s50aAZC3KSkOaEvXawC3z1KS', thread_id='thread_1lstmo59ECaxNsaicn2zlDsu'), ThreadMessage(id='msg_mOCMfgpKfLFcGA5Id7CnTKyb', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the date of birth of Zia?'), type='text')], created_at=1703524490, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, th

## Deleting files

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file.id
)

# Detaching the file from the assistant removes the file from the retrieval index and means you will no longer be charged for the storage of the indexed file.